In [2]:
import random
import numpy as np
from scipy.io.wavfile import write
import IPython.display as ipd
import math

In [4]:
import random
import numpy as np
from scipy.io.wavfile import write
import IPython.display as ipd
import math

def melody_generation(tone=440, key='minor', duration=60, bpm=120, wave_type='sine', sample_rate=44100):
    
    ## Создание цепочки аккордов для минорной и мажорной тональностей (по тонам):
    if key == 'minor':
        second, third = random.sample([3, 4, 6, 7], 2)
        chords_chain = [1, second, third, 5] 
    elif key == 'major':
        second, third = random.sample([3, 4, 6, 7], 2)
        chords_chain = [1, second, third, 5]
    
    ## Преобразование ступеней цепочки в частоты:
    major_intervals = [0, 2, 4, 5, 7, 9, 11]
    minor_intervals = [0, 2, 3, 5, 7, 8, 10]

    def note_frequency(tone, step):
        a = 2 ** (1 / 12)
        return tone * (a ** step)

    second = chords_chain[1] - 1
    third = chords_chain[2] - 1
    dominant = note_frequency(tone, 7)
    
    if key == 'major':
        second = major_intervals[second]
        third = major_intervals[third]
    elif key == 'minor':
        second = minor_intervals[second]
        third = minor_intervals[third]      

    second = note_frequency(tone, second)
    third = note_frequency(tone, third)
    
    frequencies = [tone, second, third, dominant]
    
    ## Сравнение частот цепочки и транспонирование при большом расстоянии
    def adjust_to_tone(value, tone):
        best_value = value
        smallest_distance = abs(value - tone)
        divided_value = value
        multiplied_value = value
        
        while divided_value >= tone or multiplied_value <= tone * 2:
            divided_value /= 2
            multiplied_value *= 2
            
            for val in [divided_value, multiplied_value]:
                distance = abs(val - tone)
                if distance < smallest_distance:
                    best_value = val
                    smallest_distance = distance
                    
        return best_value
    
    for i in range(1, len(frequencies)):
        frequencies[i] = adjust_to_tone(frequencies[i], tone)

    ## Создание звуковой волны для цепочки аккордов:
    chain_duration = 4 / (bpm / 60)
    time = np.linspace(0, chain_duration, int(sample_rate * chain_duration), endpoint=False)

    def sine(frequency, amplitude=0.5):
        return amplitude * np.sin(2 * np.pi * frequency * time)
    def saw(frequency, amplitude=0.5):
        period = 1 / frequency
        phase = (time / period) % 1
        return (amplitude * 2 * phase) - amplitude
    def square(frequency, amplitude=0.5):
        return amplitude * np.sign(np.sin(2 * np.pi * frequency * time))
    def triangle(frequency, amplitude=0.5):
        return amplitude * (2 * np.abs(2 * (time * frequency - np.floor(time * frequency + 0.5))) - 1)

    wave_functions = {
        'sine': sine,
        'saw': saw,
        'square': square,
        'triangle': triangle
    }

    def major_triad(tone, chord_duration):
        mediant = note_frequency(tone, 4)
        dominant = note_frequency(tone, 7)
        frequencies = [tone, mediant, dominant]
        waves = [wave_functions[wave_type](freq) for freq in frequencies]
        wave = np.sum(waves, axis=0)
        wave /= np.max(np.abs(wave))
        wave = np.int16(wave * 32767)
        return wave

    def minor_triad(tone, chord_duration):
        mediant = note_frequency(tone, 4)
        dominant = note_frequency(tone, 7)
        frequencies = [tone, mediant, dominant]
        waves = [wave_functions[wave_type](freq) for freq in frequencies]
        wave = np.sum(waves, axis=0)
        wave /= np.max(np.abs(wave))
        wave = np.int16(wave * 32767)
        return wave

    def create_chord_wave(tone, duration, key, chords_chain):
        i = len(chords_chain)
        chords = chords_chain
        chord_waves = []

        if key == 'major':
            chord_waves.append(major_triad(tone, duration / i))
            chord_waves.append(major_triad(note_frequency(tone, chords[1]), duration / i))
            chord_waves.append(major_triad(note_frequency(tone, chords[2]), duration / i))
            chord_waves.append(major_triad(note_frequency(tone, chords[3]), duration / i))
        else:
            chord_waves.append(minor_triad(tone, duration / i))
            chord_waves.append(minor_triad(note_frequency(tone, chords[1]), duration / i))
            chord_waves.append(minor_triad(note_frequency(tone, chords[2]), duration / i))
            chord_waves.append(minor_triad(note_frequency(tone, chords[3]), duration / i))

        chain_wave = np.concatenate(chord_waves)
        accompaniment = np.tile(chain_wave, duration // int(chain_duration))

        remaining_samples = int((duration % chain_duration) * sample_rate)
        if remaining_samples > 0:
            accompaniment = np.concatenate((accompaniment, chain_wave[:remaining_samples]))
        
        write('accompaniment.wav', sample_rate, accompaniment)
        return accompaniment

    ## Создание мелодии:
    def rhythmic_pattern():
        values = [0.125, 0.25, 0.5, 0.75, 1]
        weights = [5, 4, 3, 2, 1]  
        pattern = []
        current_sum = 0
        
        while current_sum < 1:
            value = random.choices(values, weights)[0]
            if current_sum + value <= 1:
                pattern.append(value)
                current_sum += value
        random.shuffle(pattern)
        return pattern

    def chord_step(start):
        values = []
        current_value = start
        for _ in range(3):
            values.append(current_value)
            current_value += 2
            if current_value > 7:
                current_value = current_value % 7  
                if current_value == 0:
                    current_value = 7
        return values
    
    def chord_st_chain(input_list):
        results = {}
        for i, value in enumerate(input_list):
            results[f"ch_{i+1}"] = chord_step(value)
        return results
        
    result = chord_st_chain(chords_chain)

    def generate_melody(chord_st_chain):
        all_steps = []
        all_durations = []
    
        for notes in chord_st_chain.values():
            rythm = rhythmic_pattern()
            melody_notes = [random.choice(notes) for _ in rythm]
            all_steps.extend(melody_notes)
            all_durations.extend(rythm)
        
        return [all_steps, all_durations]

    melody = generate_melody(result)

    def melody_freq(melody, tone):
        if key == 'major':
            intervals = [0, 2, 4, 5, 7, 9, 11]
        else:
            intervals = [0, 2, 3, 5, 7, 8, 10]
            
        steps = melody[0]
        durations = melody[1]
        frequencies = [note_frequency(tone, intervals[step - 1]) for step in steps]
        return [frequencies, durations]
    
    melody_frequencies = melody_freq(melody, tone)

    def create_melody_wave(melody_frequencies, duration, sample_rate):
        frequencies, durations = melody_frequencies
        melody_wave = np.array([])

        for freq, dur in zip(frequencies, durations):
            t = np.linspace(0, dur, int(sample_rate * dur), endpoint=False)
            wave = wave_functions[wave_type](freq)
            melody_wave = np.concatenate((melody_wave, wave))

        num_repeats = duration // int(chain_duration)
        remaining_time = duration % int(chain_duration)
        remaining_samples = int(remaining_time * sample_rate)
        
        melody_wave = np.tile(melody_wave, num_repeats)
        if remaining_samples > 0:
            melody_wave = np.concatenate((melody_wave, melody_wave[:remaining_samples]))
        
        write('melody.wav', sample_rate, melody_wave)
        return melody_wave

    accompaniment = create_chord_wave(tone, duration, key, chords_chain)
    melody = create_melody_wave(melody_frequencies, duration, sample_rate)

   


NameError: name 'accompaniment' is not defined

In [5]:
melody_generation(tone=440, key='minor', duration=60, bpm=120, wave_type='sine', sample_rate=44100)
ipd.Audio('melody.wav')

NameError: name 'accompaniment' is not defined